In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership","annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths","open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt","total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d","collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il","open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc","all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy","bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc","mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl","num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m","num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit","total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df_dtypes = pd.DataFrame(df.dtypes)

In [7]:
df_dtypes.index = df_dtypes.index.set_names(['Column_Name'])
df_dtypes.rename(columns = {0: "Data_Type"}, inplace=True)
df_dtypes.reset_index()

,Column_Name,Data_Type
0,loan_amnt,float64
1,int_rate,float64
2,installment,float64
3,home_ownership,object
4,annual_inc,float64
...,...,...
81,total_bal_ex_mort,float64
82,total_bc_limit,float64
83,total_il_high_credit_limit,float64
84,hardship_flag,object


In [8]:
df_string = df_dtypes[df_dtypes['Data_Type']=='object']
df_string

,Data_Type
Column_Name,
home_ownership,object
verification_status,object
issue_d,object
loan_status,object
pymnt_plan,object
initial_list_status,object
next_pymnt_d,object
application_type,object
hardship_flag,object


In [9]:
# Create the training variables by converting the string values into numerical ones using the get_dummies() method.
df = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan", "initial_list_status","next_pymnt_d",
                                                  "application_type", "hardship_flag", "debt_settlement_flag"])
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [10]:
# Create our features
X = df.drop(columns="loan_status")
# Create our target
y = df["loan_status"]

In [11]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

In [14]:
from sklearn.preprocessing import StandardScaler
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [16]:
# Display the confusion matrix
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
# Calculating the confusion matrix.
rf_cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
rf_cm_df = pd.DataFrame(rf_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
rf_cm_df

,Predicted 0,Predicted 1
Actual 0,71,30
Actual 1,2146,14958


In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
rf_bal_acc_score= balanced_accuracy_score(y_test, predictions)
rf_bal_acc_score

0.7887512850910909

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
rf_class_rpt=(classification_report_imbalanced(y_test, predictions))
print(rf_class_rpt)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205



In [19]:
from sklearn.metrics import recall_score
rf_recall_0=recall_score(y_test, predictions, average="binary",pos_label="high_risk")
rf_recall_1=recall_score(y_test, predictions, average="binary",pos_label="low_risk")

In [20]:
from sklearn.metrics import precision_score
rf_precision_0=precision_score(y_test, predictions, average="binary",pos_label="high_risk")
rf_precision_1=precision_score(y_test, predictions, average="binary",pos_label="low_risk")

In [21]:
# Displaying all results
print("Confusion Matrix")
display(rf_cm_df)
print(f"Balanced Accuracy Score : {rf_bal_acc_score}")
print("Classification Report")
print(rf_class_rpt)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,71,30
Actual 1,2146,14958


Balanced Accuracy Score : 0.7887512850910909
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205



In [22]:
pd.options.display.max_rows = 100
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sort_orders = sorted(zip(X.columns,importances),reverse=False)
feature_importances = pd.DataFrame(sort_orders,  columns=['Features','importance']).sort_values('importance', ascending=False)
feature_importances

,Features,importance
90,total_rec_prncp,0.078768
86,total_pymnt,0.058838
87,total_pymnt_inv,0.056256
88,total_rec_int,0.053555
32,last_pymnt_amnt,0.050033
28,int_rate,0.029670
30,issue_d_Jan-2019,0.021129
27,installment,0.019802
15,dti,0.017471
67,out_prncp_inv,0.016858


### Easy Ensemble AdaBoost Classifier

In [23]:
from collections import Counter
from sklearn.datasets import make_classification
# Train the EasyEnsembleClassifier

from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1) 
# Fitting the model
eec_model = eec_model.fit(X_train_scaled, y_train)

In [24]:
# Display the confusion matrix
# Making predictions using the testing data.
predictions = eec_model.predict(X_test_scaled)
# Calculating the confusion matrix.
ee_cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
ee_cm_df = pd.DataFrame(ee_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
ee_cm_df

,Predicted 0,Predicted 1
Actual 0,93,8
Actual 1,985,16119


In [25]:
# Calculated the balanced accuracy score
ee_bal_acc_score= balanced_accuracy_score(y_test, predictions)

In [26]:
# Print the imbalanced classification report
#ee_class_rpt=(classification_report_imbalanced(y_test, predictions, output_dict=True))
ee_class_rpt=(classification_report_imbalanced(y_test, predictions))
print(ee_class_rpt)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



In [27]:
from sklearn.metrics import recall_score
eec_recall_0=recall_score(y_test, predictions, average="binary",pos_label="high_risk")
eec_recall_1=recall_score(y_test, predictions, average="binary",pos_label="low_risk")

In [28]:
from sklearn.metrics import precision_score
eec_precision_0=precision_score(y_test, predictions, average="binary",pos_label="high_risk")
eec_precision_1=precision_score(y_test, predictions, average="binary",pos_label="low_risk")

In [29]:
# Displaying all results
print("Confusion Matrix")
display(ee_cm_df)
print(f" Balanced Accuracy Score : {ee_bal_acc_score}")
print("Classification Report")
print(ee_class_rpt)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,93,8
Actual 1,985,16119


 Balanced Accuracy Score : 0.931601605553446
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



# Comparing Model Performance

## Balanced Random Forest Classifier and Easy Ensemble AdaBoost Classifier

In [30]:
# Create DataFrame for the balanced accuracy score across the 2 models
Balanced_Accuracy_Score_df = pd.DataFrame(
          [{"Random Forest": round(rf_bal_acc_score,2), "EasyEnsemble Classifier": round(ee_bal_acc_score,2)
          }], index=['Balanced_Accuracy_Score'])
Balanced_Accuracy_Score_df

,Random Forest,EasyEnsemble Classifier
Balanced_Accuracy_Score,0.79,0.93


In [31]:
# Create DataFrame for the recall and precision scores across the 2 models
dict =  {'Rf_Recall': [round(rf_recall_0,2), round(rf_recall_1,2)], 'Rf_Precision': [round(rf_precision_0,2), round(rf_precision_1,2)],
        'Eec_Recall': [round(eec_recall_0,2), round(eec_recall_1,2)], 'Eec_Precision': [round(eec_precision_0,2), round(eec_precision_1,2)]
        }
idx = ['high_risk', 'low_risk']
Recall_Precsion_df = pd.DataFrame(dict,index=idx)
Recall_Precsion_df

,Rf_Recall,Rf_Precision,Eec_Recall,Eec_Precision
high_risk,0.70,0.03,0.92,0.09
low_risk,0.87,1.00,0.94,1.00
